In [2]:
import os
from collections import defaultdict

def contar_palavras(texto):
    return len(texto.split())

def listar_notas_markdown_organizadas(pasta_raiz):
    notas_por_pasta = defaultdict(list)
    contagem_palavras_por_pasta = defaultdict(int)
    total_palavras_geral = 0

    for raiz, _, arquivos in os.walk(pasta_raiz):
        caminho_relativo = os.path.relpath(raiz, pasta_raiz)
        for arquivo in arquivos:
            if arquivo.endswith(".md"):
                caminho_completo = os.path.join(raiz, arquivo)
                nome_nota = os.path.splitext(arquivo)[0]
                
                with open(caminho_completo, 'r', encoding='utf-8') as f:
                    conteudo = f.read()
                    palavras = contar_palavras(conteudo)
                    total_palavras_geral += palavras
                    contagem_palavras_por_pasta[caminho_relativo] += palavras
                    notas_por_pasta[caminho_relativo].append((nome_nota, palavras))

    return notas_por_pasta, contagem_palavras_por_pasta, total_palavras_geral

def salvar_em_markdown(notas_organizadas, contagem_por_pasta, total_geral, caminho_saida):
    with open(caminho_saida, 'w', encoding='utf-8') as f:
        f.write(f"# Índice de Notas Markdown\n\n")
        for pasta, notas in sorted(notas_organizadas.items()):
            titulo_pasta = pasta if pasta != '.' else '[raiz]'
            f.write(f"### {titulo_pasta} — {len(notas)} notas\n")
            for nome_nota, palavras in sorted(notas):
                f.write(f"- [{nome_nota}] — {palavras} palavras\n")
            f.write(f"\n**Total nesta pasta**: {contagem_por_pasta[pasta]} palavras\n\n")

        f.write("---\n")
        f.write(f"## 📊 Total geral: {total_geral} palavras\n")

# Caminhos
caminho_da_pasta = r"C:\Users\nonak\Documents\Thoughts"
caminho_arquivo_saida = os.path.join(caminho_da_pasta, "_index_notas.md")

# Execução
notas_organizadas, contagem_por_pasta, total_palavras = listar_notas_markdown_organizadas(caminho_da_pasta)
salvar_em_markdown(notas_organizadas, contagem_por_pasta, total_palavras, caminho_arquivo_saida)

print(f"Arquivo salvo em: {caminho_arquivo_saida}")


Arquivo salvo em: C:\Users\nonak\Documents\Thoughts\_index_notas.md


In [2]:
import os
from collections import defaultdict
from pathlib import Path
import time
from typing import Dict, List, Tuple

def contar_palavras(texto: str) -> int:
    """Conta palavras em uma string, ignorando linhas que são links Markdown."""
    palavras = 0
    for linha in texto.splitlines():
        linha = linha.strip()
        # Ignora linhas que são apenas links Markdown
        if not (linha.startswith("[") and "]" in linha and "(" in linha and linha.endswith(")")):
            palavras += len(linha.split())
    return palavras

def listar_notas_markdown_organizadas(pasta_raiz: str) -> Tuple[Dict[str, List[Tuple[str, int]]], Dict[str, int], int]:
    """
    Organiza notas Markdown por pasta e conta palavras.
    
    Retorna:
        - Dicionário de notas por pasta
        - Contagem de palavras por pasta
        - Total geral de palavras
    """
    notas_por_pasta = defaultdict(list)
    contagem_palavras_por_pasta = defaultdict(int)
    total_palavras_geral = 0
    total_notas = 0

    for raiz, _, arquivos in os.walk(pasta_raiz):
        caminho_relativo = os.path.relpath(raiz, pasta_raiz)
        for arquivo in sorted(arquivos):  # Ordena os arquivos alfabeticamente
            if arquivo.lower().endswith(".md") and not arquivo.startswith("_"):  # Ignora arquivos que começam com _
                caminho_completo = os.path.join(raiz, arquivo)
                nome_nota = os.path.splitext(arquivo)[0]
                
                try:
                    with open(caminho_completo, 'r', encoding='utf-8') as f:
                        conteudo = f.read()
                        palavras = contar_palavras(conteudo)
                        total_palavras_geral += palavras
                        contagem_palavras_por_pasta[caminho_relativo] += palavras
                        notas_por_pasta[caminho_relativo].append((nome_nota, palavras, caminho_completo))
                        total_notas += 1
                except Exception as e:
                    print(f"Erro ao processar {caminho_completo}: {str(e)}")

    print(f"\nProcessamento concluído: {total_notas} notas encontradas.")
    return notas_por_pasta, contagem_palavras_por_pasta, total_palavras_geral

def formatar_numero(num: int) -> str:
    """Formata números grandes com separadores de milhar."""
    return f"{num:,}".replace(",", ".")

def salvar_em_markdown(notas_organizadas: Dict[str, List[Tuple[str, int]]], 
                      contagem_por_pasta: Dict[str, int], 
                      total_geral: int, 
                      caminho_saida: str) -> None:
    """
    Salva o índice organizado em um arquivo Markdown com formatação melhorada.
    """
    with open(caminho_saida, 'w', encoding='utf-8') as f:
        # Cabeçalho com metadados
        f.write(f"""---
# Índice Automático de Notas

- **Data de geração**: {time.strftime("%d/%m/%Y %H:%M:%S")}
- **Total de notas**: {sum(len(notas) for notas in notas_organizadas.values())}
- **Total de palavras**: {formatar_numero(total_geral)}
---

""")
        
        # Sumário rápido
        f.write("## 📋 Sumário\n\n")
        for pasta in sorted(notas_organizadas.keys()):
            titulo_pasta = pasta if pasta != '.' else 'Raiz'
            num_notas = len(notas_organizadas[pasta])
            palavras_pasta = contagem_por_pasta[pasta]
            f.write(f"- **{titulo_pasta}**: {num_notas} notas ({formatar_numero(palavras_pasta)} palavras)\n")
        f.write("\n---\n\n")
        
        # Detalhes por pasta
        f.write("## 📂 Notas por Pasta\n\n")
        for pasta, notas in sorted(notas_organizadas.items()):
            titulo_pasta = pasta if pasta != '.' else '[Raiz]'
            num_notas = len(notas)
            palavras_pasta = contagem_por_pasta[pasta]
            
            f.write(f"### {titulo_pasta}\n")
            f.write(f"*{num_notas} notas • {formatar_numero(palavras_pasta)} palavras • {palavras_pasta/num_notas:.1f} palavras/nota*\n\n")
            
            for nome_nota, palavras, caminho in sorted(notas, key=lambda x: x[0].lower()):
                caminho_rel = os.path.relpath(caminho, os.path.dirname(caminho_saida))
                f.write(f"- [{nome_nota}]({caminho_rel}) — {formatar_numero(palavras)} palavras\n")
            
            f.write("\n")

        # Estatísticas finais
        f.write("---\n\n")
        f.write("## 📊 Estatísticas\n\n")
        f.write(f"- **Total de notas**: {sum(len(notas) for notas in notas_organizadas.values())}\n")
        f.write(f"- **Total de palavras**: {formatar_numero(total_geral)}\n")
        media_geral = total_geral / sum(len(notas) for notas in notas_organizadas.values()) if sum(len(notas) for notas in notas_organizadas.values()) > 0 else 0
        f.write(f"- **Média geral**: {media_geral:.1f} palavras/nota\n")

def main():
    # Configuração usando Path para melhor manipulação de caminhos
    caminho_da_pasta = Path.home() / "Documents" / "Thoughts"
    caminho_arquivo_saida = caminho_da_pasta / "_index_notas4.md"
    
    print(f"Processando notas em: {caminho_da_pasta}")
    
    # Execução
    notas_organizadas, contagem_por_pasta, total_palavras = listar_notas_markdown_organizadas(str(caminho_da_pasta))
    salvar_em_markdown(notas_organizadas, contagem_por_pasta, total_palavras, str(caminho_arquivo_saida))
    
    print(f"\n✅ Arquivo de índice gerado com sucesso em:\n{caminho_arquivo_saida}")

if __name__ == "__main__":
    main()

Processando notas em: C:\Users\nonak\Documents\Thoughts

Processamento concluído: 1331 notas encontradas.

✅ Arquivo de índice gerado com sucesso em:
C:\Users\nonak\Documents\Thoughts\_index_notas4.md
